In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 23) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 25) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 27) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [23, 25, 27]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [7]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.417480230331421, Final Batch Loss: 2.203195095062256
Epoch 2, Loss: 4.4155333042144775, Final Batch Loss: 2.214625835418701
Epoch 3, Loss: 4.410343408584595, Final Batch Loss: 2.1888551712036133
Epoch 4, Loss: 4.402981758117676, Final Batch Loss: 2.189234972000122
Epoch 5, Loss: 4.395047903060913, Final Batch Loss: 2.2000443935394287
Epoch 6, Loss: 4.392462730407715, Final Batch Loss: 2.189796209335327
Epoch 7, Loss: 4.3924713134765625, Final Batch Loss: 2.191723108291626
Epoch 8, Loss: 4.386497497558594, Final Batch Loss: 2.1944785118103027
Epoch 9, Loss: 4.378002882003784, Final Batch Loss: 2.1860768795013428
Epoch 10, Loss: 4.372597932815552, Final Batch Loss: 2.187197685241699
Epoch 11, Loss: 4.366967439651489, Final Batch Loss: 2.158565044403076
Epoch 12, Loss: 4.362393617630005, Final Batch Loss: 2.18190598487854
Epoch 13, Loss: 4.357719898223877, Final Batch Loss: 2.1699423789978027
Epoch 14, Loss: 4.340387582778931, Final Batch Loss: 2.166548728942871
Epoch 15,

Epoch 127, Loss: 2.0656813383102417, Final Batch Loss: 0.9936341047286987
Epoch 128, Loss: 2.101752281188965, Final Batch Loss: 1.0237901210784912
Epoch 129, Loss: 2.0289372205734253, Final Batch Loss: 1.007084608078003
Epoch 130, Loss: 2.110980272293091, Final Batch Loss: 1.0902070999145508
Epoch 131, Loss: 2.1069328784942627, Final Batch Loss: 1.0688987970352173
Epoch 132, Loss: 2.0414472818374634, Final Batch Loss: 1.0075476169586182
Epoch 133, Loss: 2.0814130306243896, Final Batch Loss: 1.066582441329956
Epoch 134, Loss: 2.093071937561035, Final Batch Loss: 1.050696611404419
Epoch 135, Loss: 2.0031570196151733, Final Batch Loss: 0.9957283735275269
Epoch 136, Loss: 2.065882444381714, Final Batch Loss: 1.0061935186386108
Epoch 137, Loss: 2.0617074966430664, Final Batch Loss: 1.0420551300048828
Epoch 138, Loss: 2.0017667412757874, Final Batch Loss: 0.9853231310844421
Epoch 139, Loss: 2.0015493631362915, Final Batch Loss: 1.0143135786056519
Epoch 140, Loss: 2.1015809774398804, Final Ba

Epoch 239, Loss: 1.3689220547676086, Final Batch Loss: 0.6552038788795471
Epoch 240, Loss: 1.3317305445671082, Final Batch Loss: 0.6627016067504883
Epoch 241, Loss: 1.339768409729004, Final Batch Loss: 0.626476526260376
Epoch 242, Loss: 1.379750669002533, Final Batch Loss: 0.6745806336402893
Epoch 243, Loss: 1.4434533715248108, Final Batch Loss: 0.7930467128753662
Epoch 244, Loss: 1.3598390817642212, Final Batch Loss: 0.711434006690979
Epoch 245, Loss: 1.4444997906684875, Final Batch Loss: 0.6954709887504578
Epoch 246, Loss: 1.3558059334754944, Final Batch Loss: 0.6632834076881409
Epoch 247, Loss: 1.2709822058677673, Final Batch Loss: 0.6138315796852112
Epoch 248, Loss: 1.3396541476249695, Final Batch Loss: 0.6736400723457336
Epoch 249, Loss: 1.3123886585235596, Final Batch Loss: 0.6411944627761841
Epoch 250, Loss: 1.3880318999290466, Final Batch Loss: 0.6797888875007629
Epoch 251, Loss: 1.3222402930259705, Final Batch Loss: 0.6054357886314392
Epoch 252, Loss: 1.2667637467384338, Final

Epoch 350, Loss: 1.045074701309204, Final Batch Loss: 0.5305483937263489
Epoch 351, Loss: 1.0662768483161926, Final Batch Loss: 0.5579493045806885
Epoch 352, Loss: 1.097410798072815, Final Batch Loss: 0.5760346055030823
Epoch 353, Loss: 1.087701678276062, Final Batch Loss: 0.5352252721786499
Epoch 354, Loss: 1.021436870098114, Final Batch Loss: 0.50831139087677
Epoch 355, Loss: 1.1161799430847168, Final Batch Loss: 0.5912328362464905
Epoch 356, Loss: 1.1757263541221619, Final Batch Loss: 0.5586245656013489
Epoch 357, Loss: 1.1771308779716492, Final Batch Loss: 0.6009085774421692
Epoch 358, Loss: 1.1748434901237488, Final Batch Loss: 0.6184618473052979
Epoch 359, Loss: 1.0931630730628967, Final Batch Loss: 0.5713986754417419
Epoch 360, Loss: 1.0408830642700195, Final Batch Loss: 0.4773048162460327
Epoch 361, Loss: 1.1155487298965454, Final Batch Loss: 0.5545969009399414
Epoch 362, Loss: 1.11543208360672, Final Batch Loss: 0.5703341960906982
Epoch 363, Loss: 1.1309600472450256, Final Bat

Epoch 475, Loss: 0.9678649008274078, Final Batch Loss: 0.46565255522727966
Epoch 476, Loss: 0.9565528631210327, Final Batch Loss: 0.4426443576812744
Epoch 477, Loss: 1.0456750690937042, Final Batch Loss: 0.48799750208854675
Epoch 478, Loss: 1.0082806646823883, Final Batch Loss: 0.5316212773323059
Epoch 479, Loss: 1.006066918373108, Final Batch Loss: 0.5073798298835754
Epoch 480, Loss: 0.9936303198337555, Final Batch Loss: 0.524957537651062
Epoch 481, Loss: 0.976846843957901, Final Batch Loss: 0.48921477794647217
Epoch 482, Loss: 1.0118446350097656, Final Batch Loss: 0.5194612741470337
Epoch 483, Loss: 1.0436438918113708, Final Batch Loss: 0.5208855271339417
Epoch 484, Loss: 1.0604056417942047, Final Batch Loss: 0.5613831281661987
Epoch 485, Loss: 0.9543739855289459, Final Batch Loss: 0.49662744998931885
Epoch 486, Loss: 0.9880968332290649, Final Batch Loss: 0.5320672988891602
Epoch 487, Loss: 1.0062196254730225, Final Batch Loss: 0.534880518913269
Epoch 488, Loss: 1.0334275364875793, F

...

Epoch 4492, Loss: 0.26691487431526184, Final Batch Loss: 0.1579217165708542
Epoch 4493, Loss: 0.25695835798978806, Final Batch Loss: 0.10259883850812912
Epoch 4494, Loss: 0.2525462731719017, Final Batch Loss: 0.14780956506729126
Epoch 4495, Loss: 0.24839933961629868, Final Batch Loss: 0.13707910478115082
Epoch 4496, Loss: 0.270177960395813, Final Batch Loss: 0.10742707550525665
Epoch 4497, Loss: 0.24816831201314926, Final Batch Loss: 0.14438849687576294
Epoch 4498, Loss: 0.20302674919366837, Final Batch Loss: 0.11275661736726761
Epoch 4499, Loss: 0.2334495633840561, Final Batch Loss: 0.12861327826976776
Epoch 4500, Loss: 0.2551723048090935, Final Batch Loss: 0.1312924474477768
Epoch 4501, Loss: 0.2627154216170311, Final Batch Loss: 0.1484857052564621
Epoch 4502, Loss: 0.26359523087739944, Final Batch Loss: 0.11172489076852798
Epoch 4503, Loss: 0.23417264223098755, Final Batch Loss: 0.12858310341835022
Epoch 4504, Loss: 0.22400157898664474, Final Batch Loss: 0.10580305010080338
Epoch 45

Epoch 4620, Loss: 0.26130450516939163, Final Batch Loss: 0.14325366914272308
Epoch 4621, Loss: 0.26717659085989, Final Batch Loss: 0.11905733495950699
Epoch 4622, Loss: 0.2677677720785141, Final Batch Loss: 0.13607630133628845
Epoch 4623, Loss: 0.27481282502412796, Final Batch Loss: 0.15628190338611603
Epoch 4624, Loss: 0.22912582755088806, Final Batch Loss: 0.14277979731559753
Epoch 4625, Loss: 0.23068930953741074, Final Batch Loss: 0.1096893772482872
Epoch 4626, Loss: 0.27216773480176926, Final Batch Loss: 0.17172546684741974
Epoch 4627, Loss: 0.18607056140899658, Final Batch Loss: 0.08690241724252701
Epoch 4628, Loss: 0.2641955614089966, Final Batch Loss: 0.16254067420959473
Epoch 4629, Loss: 0.2660805732011795, Final Batch Loss: 0.09813196957111359
Epoch 4630, Loss: 0.21956531703472137, Final Batch Loss: 0.11818527430295944
Epoch 4631, Loss: 0.27598198503255844, Final Batch Loss: 0.1233542338013649
Epoch 4632, Loss: 0.26472635567188263, Final Batch Loss: 0.1281430721282959
Epoch 46

Epoch 4745, Loss: 0.29780347645282745, Final Batch Loss: 0.1721975952386856
Epoch 4746, Loss: 0.28279344737529755, Final Batch Loss: 0.1440158486366272
Epoch 4747, Loss: 0.23226813226938248, Final Batch Loss: 0.09748228639364243
Epoch 4748, Loss: 0.2142656370997429, Final Batch Loss: 0.09301290661096573
Epoch 4749, Loss: 0.20414021611213684, Final Batch Loss: 0.11511198431253433
Epoch 4750, Loss: 0.31166674196720123, Final Batch Loss: 0.16019044816493988
Epoch 4751, Loss: 0.2231101542711258, Final Batch Loss: 0.10999378561973572
Epoch 4752, Loss: 0.23045466095209122, Final Batch Loss: 0.14454886317253113
Epoch 4753, Loss: 0.24986495077610016, Final Batch Loss: 0.14386174082756042
Epoch 4754, Loss: 0.23263638466596603, Final Batch Loss: 0.13943159580230713
Epoch 4755, Loss: 0.22811077535152435, Final Batch Loss: 0.11035322397947311
Epoch 4756, Loss: 0.20254459977149963, Final Batch Loss: 0.09834203869104385
Epoch 4757, Loss: 0.2524235099554062, Final Batch Loss: 0.10390472412109375
Epoc

Epoch 4871, Loss: 0.28348373621702194, Final Batch Loss: 0.17529547214508057
Epoch 4872, Loss: 0.2103041484951973, Final Batch Loss: 0.10014725476503372
Epoch 4873, Loss: 0.24602264910936356, Final Batch Loss: 0.1293765902519226
Epoch 4874, Loss: 0.21447765827178955, Final Batch Loss: 0.09731850028038025
Epoch 4875, Loss: 0.1875864341855049, Final Batch Loss: 0.06942155957221985
Epoch 4876, Loss: 0.22279765456914902, Final Batch Loss: 0.13438118994235992
Epoch 4877, Loss: 0.24967603385448456, Final Batch Loss: 0.11691330373287201
Epoch 4878, Loss: 0.2489570528268814, Final Batch Loss: 0.165549635887146
Epoch 4879, Loss: 0.17011752724647522, Final Batch Loss: 0.08679133653640747
Epoch 4880, Loss: 0.27698399126529694, Final Batch Loss: 0.1440456062555313
Epoch 4881, Loss: 0.24322015047073364, Final Batch Loss: 0.11607219278812408
Epoch 4882, Loss: 0.24388515204191208, Final Batch Loss: 0.156395822763443
Epoch 4883, Loss: 0.22779599577188492, Final Batch Loss: 0.12780757248401642
Epoch 48

Epoch 4995, Loss: 0.21826045215129852, Final Batch Loss: 0.10295503586530685
Epoch 4996, Loss: 0.29214294254779816, Final Batch Loss: 0.17903666198253632
Epoch 4997, Loss: 0.23645202070474625, Final Batch Loss: 0.09775232523679733
Epoch 4998, Loss: 0.22122252732515335, Final Batch Loss: 0.11259970813989639
Epoch 4999, Loss: 0.2610060125589371, Final Batch Loss: 0.158168762922287
Epoch 5000, Loss: 0.2605656087398529, Final Batch Loss: 0.13585281372070312


In [8]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[12  0  0  0  0  0  0  0  0]
 [ 0 11  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  1  0  0  0]
 [ 0  0  0 13  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  0  0]
 [ 0  0  5  0  0 12  0  0  0]
 [ 0  0  0  0  0  0 11  0  0]
 [ 0  0  0  0  0  0  0  6  0]
 [ 0  0  0  0  0  1  0  0 13]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        12
           1    1.00000   1.00000   1.00000        11
           2    0.70588   0.92308   0.80000        13
           3    1.00000   1.00000   1.00000        13
           4    1.00000   1.00000   1.00000        13
           5    0.85714   0.70588   0.77419        17
           6    1.00000   1.00000   1.00000        11
           7    1.00000   1.00000   1.00000         6
           8    1.00000   0.92857   0.96296        14

    accuracy                        0.93636       110
   macro avg    0.95145   0.95084   0.94857       110
weighted avg    0.94316   0.93636   0.93675       110



In [37]:
n_samples = 30

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples) + 1

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_4 = np.ones(n_samples) + 2

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_5 = np.ones(n_samples) + 3

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_6 = np.ones(n_samples) + 4

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_7 = np.ones(n_samples) + 5

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_8 = np.ones(n_samples) + 6

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_9 = np.ones(n_samples) + 7

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
fake_labels = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

fake_features = torch.Tensor(fake_features)

In [38]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[26  0  0  2  0  0  2  0  0]
 [ 0 30  0  0  0  0  0  0  0]
 [ 0  0 29  0  0  1  0  0  0]
 [ 0  0  0 29  1  0  0  0  0]
 [ 0  0  0  0 23  0  0  7  0]
 [ 0  0  6  0  0 19  0  0  5]
 [ 3  1  0  0  0  0 26  0  0]
 [ 0  0  0  0  0  0  0 30  0]
 [ 0  0 11  0  0  1  0  0 18]]
              precision    recall  f1-score   support

         0.0    0.89655   0.86667   0.88136        30
         1.0    0.96774   1.00000   0.98361        30
         2.0    0.63043   0.96667   0.76316        30
         3.0    0.93548   0.96667   0.95082        30
         4.0    0.95833   0.76667   0.85185        30
         5.0    0.90476   0.63333   0.74510        30
         6.0    0.92857   0.86667   0.89655        30
         7.0    0.81081   1.00000   0.89552        30
         8.0    0.78261   0.60000   0.67925        30

    accuracy                        0.85185       270
   macro avg    0.86837   0.85185   0.84969       270
weighted avg    0.86837   0.85185   0.84969       270

